# Python and SQL
Now that we're comfortable using SQL, let return to Python and learn how it can interact with an SQL database. Why do we want to do this? Well, as we saw in Chapter 1, Python can be used to clean data and perform other complex calculations so being able to read data from an SQL database and write the output back to a database allows us to combine the power of Python as a programming language with the data storage of a SQL database. 

In this episode, we will learn how to connect to a database using the Python library SQLAlchemy, how use Pandas to read and write data from our database, and finally see how we can directly query our data with SQLAlchemy. 

## Setup
In Chapter 1, we installed Pandas in our Python virtual environment. We will do this again and include SQLAlchemy by running the `setup.sh` script from this episode. The associated `requirements.txt` includes both Pandas and SQLAlchemy, and supporting library PyMySQL. That will run the following: 

In [ ]:
%%bash

# update pip
python3.7 -m pip install --upgrade pip setuptools wheel virtualenv

# create and run a python3.7 virtual env
python3.7 -m venv venv
source venv/bin/activate

# install and setup jupyter notebooks 
pip3 install jupyterlab notebook jupyterthemes
# change default theme and fonts
jt -t onedork -T -tf sourcesans -nf sourcesans -tfs 12

# pip install pypi packages
pip3 install -r requirements.txt

We then need to start up our database using the familiar `start_db.sh` script. Once the container is up and running, we're ready to learn how to connect to a database using SQLAlchemy.

# SQLAlchemy Engine
In order to access our data in the Docker MariaDB we just started, we need to use a SQLAlchemy **Engine**. Their documentation gives [a great overview of this usage which we will follow ](https://docs.sqlalchemy.org/en/13/core/connections.html). This **Engine** is a Python object that stores the information about our database and can create individual **Connections** to access the database. In order to create an engine, we simply import and use the `create_engine` function:

In [ ]:
from sqlalchemy import create_engine

`create_engine` takes a string of the following format:
`connection_method://db_user:db_password@db_host/db_name`. 
- **connection_method**: This tells SQLAlchemy *how* to connect to the database. This library support a large number of different database types so we need to specify how we connect. We will use the method `mysql+pymysql` (now you know why `PyMySQL` was included in the requirements.txt). This library works well with our MariaDB instance. 
- **db_user**: this is the user name we want to use when connecting. The default MariaDB container we're using let's us access this as `root` user.
- **db_password**: the password for our user. We set this to `mysql` in the `start_db.sh` script.
- **db_host**: the address of our database. This is set to `127.0.0.11` in `start_db.sh` so we will use that. SQLAlchemy will allow you to connect to any url that is a valid database host.
- **db_name**: the name of the database we want to access. Let's continue working with `northwind_small`.

Putting these all together, we can make our first engine:


In [ ]:
connection_method = "mysql+pymysql"
db_user = "root"
db_password = "mysql"
db_host = "127.0.0.1"
db_name = "northwind_small"

engine = create_engine(f"{connection_method}://{db_user}:{db_password}@{db_host}/{db_name}", future=True)


The `create_engine` returns an `Engine` object which itself has a `connection()` method. As the name implies, this is how we actually connect to our database. This is very similar to how Python's basic `open()` function returns a connection to a local file. To check that our Engine is able to connect, we can import `sqlalchemy.text` and give it a simple text query to execute (Don't worry about the details now, we will learn more about this soon):

In [ ]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM orders LIMIT 3"))
    for row in result:
        print(row)

# Pandas and SQL
## Reading Data
Now that we know how to connect to our database using SQLAlchemy, let's see how we can combine that with Pandas to read data from our database into a DataFrame. Pandas provides us with [the handy `read_sql` method](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) for just this purpose. It reads in data from an SQL database and returns a Pandas' DataFrame object which we are familiar with using from Chapter

In [ ]:
from pandas import read_sql

As we can see in the linked documentationThis function has two required arguments:
- **sql**: the SQL query to execute *or* a table name
- **con**: a SQLAlchemy connection; isn't it handy we just learned how to make one of these?

Let's use this function to read in the **orders** table. As noted, we simply supply our connection and the desired table name:

In [ ]:
conn = engine.connect()
orders_df = read_sql('orders', conn)
orders_df.head()
conn.close()

As noted by the documentation, we can also provide a query as the first argument in order to get more specific data. For example, let's say we wanted order information joined with customer and order details as seen in the last episode:

In [ ]:
join_query = "SELECT o.id, o.status, c.last_name customer_name, e.last_name employee_name, o.notes \
    FROM orders AS o \
    LEFT JOIN employees e ON o.employee_id = e.id \
    LEFT JOIN customers c on o.customer_id = c.id;"


This query will be executed by our connection. In order to let our SQLAlchemy know this is a text query to execute, we use the `text` function we saw earlier. This returns a SQLAlchemy **TextClause** which lets our engine know that this is a text query for it to execute on our database:

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    query_df = read_sql(text(join_query), conn)
query_df.head()

Using either pattern with `read_sql` returns our data in a familiar DataFrame ready for us to use just as we have before! Let's say we want to rename the `id` column in our `query_df` to `order_id:


In [ ]:
query_df.rename(columns={"id": "order_id"}, inplace=True)
query_df.head()

# Write 
Once we have modified our data, we may want to save the updated data in our database. As a small example, our `query_df` joins three tables and updated a column name and we might need to access this in the future. To do this, we use [the DataFrame's `to_sql()` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html). The required arguments for this are:
- **name**: the name of the table to write our data to.
- **con**: the connection to use. For writing, we need to pass in the whole SQLAlchemy engine, not just a single connection so it has the full functionality of the engine.

Additionally, we have other arguments we can set.
- **if_exists** tells our engine what to do if the table specified by **name** already exists in our database using the **if_exists** argument. By default, this is set to `'fail'` so our function will return an error if we attempt to write to an existing table. The other options are: `'replace'` - drop the table and write new values, `'append'` - append the data to the existing table. Note, this can cause errors if there are duplicates on the primary key column.
- **index**: tells Pandas whether to include the index as a column in our output table. We will set this to `False` when our data has an existing index column, or we can set that column as the DataFrame index.
- **schema**: allows the specification of the schema, or database name. While this is already set in our engine, we can use `northwind_small` as our value.
For our purposes, we can use replace to  create a new table called **order_status** which we will drop and overwrite every time the following cell is executed:

In [ ]:
new_table = "order_status"

query_df.to_sql(new_table, engine, if_exists='replace', index=False, schema='northwind_small')


And let's quickly check out output to see what we just wrote:

In [ ]:
with engine.connect() as conn:
    check = read_sql(new_table, conn)
check.head()

And now we know how to get data from a database into and out of Pandas DataFrames using SQLAlchemy!